<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports-and-definitions" data-toc-modified-id="Imports-and-definitions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports and definitions</a></span></li><li><span><a href="#Load-tables" data-toc-modified-id="Load-tables-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load tables</a></span></li><li><span><a href="#Text-association" data-toc-modified-id="Text-association-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Text association</a></span></li></ul></div>

# Analysis 2 - Sentiment

## Imports and definitions

In [1]:
%load_ext memory_profiler

In [2]:
import warnings
import datetime as dt
import re
from collections import Counter

import numpy as np
import statsmodels.api as sm
import pandas as pd
pd.set_option('max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import matplotlib.pyplot as plt
import seaborn as sb
plt.style.use('fivethirtyeight') #Generally nice
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import NMF

from pandasql import sqldf
sql = lambda x: sqldf(x, globals())
from textblob import TextBlob
path = 'output_data/'

## Load tables

In [3]:
tweet = pd.read_parquet(path+'cleantweets.parquet')

In [4]:
user = pd.read_csv(path+'user.csv')
topwords = pd.read_csv(path+'topwords.csv')
#hashtag = pd.read_csv(path+'hashtag.csv')
#mention = pd.read_csv(path+'mention.csv')
#tweet_hashtag = pd.read_csv(path+'tweet_hashtag.csv')
#tweet_mention = pd.read_csv(path+'tweet_mention.csv')

In [5]:
topwords = topwords.iloc[0:5000,:]

## Text association

In [6]:
%%time
#Set up the incidence matrix
inc_mat=CountVectorizer(vocabulary=topwords.word)
X = inc_mat.fit_transform(tweet.wordtext)

CPU times: user 7.34 s, sys: 46.6 ms, total: 7.39 s
Wall time: 7.39 s


In [7]:
%%time
# Feature extraction
n_components=8
nmf = NMF(n_components=n_components, solver='mu')
W = nmf.fit_transform(X)

CPU times: user 44.1 s, sys: 23.7 s, total: 1min 7s
Wall time: 17.3 s


In [8]:
nwords = 12
tw = np.array(inc_mat.get_feature_names())
for i, topic in enumerate(nmf.components_):
    terms = [str(term) for term in tw[topic.argsort()[-nwords:]]]
    terms = ', '.join(terms)
    print(f'Cluster {i+1} top {nwords} words: {terms}')

Cluster 1 top 12 words: proud, time, join, ago, meeting, years, us, pm, honor, office, thanks, today
Cluster 2 top 12 words: proud, people, act, need, congress, must, thanks, support, time, help, work, us
Cluster 3 top 12 words: yesterday, school, discussion, visit, meet, news, students, morning, see, meeting, thanks, great
Cluster 4 top 12 words: pass, committee, white, gop, bipartisan, act, vote, floor, senate, passed, bill, house
Cluster 5 top 12 words: law, plan, mental, coverage, women, insurance, affordable, access, act, americans, care, health
Cluster 6 top 12 words: news, check, center, create, happy, report, congrats, york, state, year, jobs, new
Cluster 7 top 12 words: first, veterans, women, honor, national, service, every, birthday, th, happy, thank, day
Cluster 8 top 12 words: et, join, listen, tonight, floor, morning, discuss, tune, pm, hearing, live, watch


* Cluster 1: general thanks for political support over time
* Cluster 2: support for American jobs
* Cluster 3: schools and education discussions
* Cluster 4: reporting on legislation and congress debates
* Cluster 5: access to healthcare and mental health
* Cluster 6: New York jobs and services
* Cluster 7: support for military veterans
* Cluster 8: drumming up audiences for political shows